In [1]:
import os
os.system("git clone https://github.com/soerenetler/squad-question-generation.git")
os.system("git clone https://github.com/soerenetler/spacy_target_answer.git")
os.system("git clone https://github.com/soerenetler/crf_target_answer.git")

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!(cd squad-question-generation/; git pull)
!(cd crf_target_answer/; git pull)

Already up to date.
Already up to date.


In [3]:
!pip install -r crf_target_answer/requirements.txt --quiet

     |████████████████████████████████| 4.9MB 26.9MB/s 
     |████████████████████████████████| 112kB 50.2MB/s 
     |████████████████████████████████| 747kB 44.8MB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.
ERROR: eli5 0.11.0 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.19.2 which is incompatible.


In [4]:
#!pip install -U scikit-learn

In [5]:
import joblib
import eli5
import pandas as pd
import ast
import spacy
from sklearn_crfsuite import metrics
from crf_target_answer.model.evaluation_utils import visualize_transitions, print_annotated_text, crf_roc_curve

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:17: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, defaultdict


ImportError: ignored

In [82]:
import json

data_folder = "/content/gdrive/MyDrive/mt-qg-data/01_data/preprocessedData/"
dataset = "squad"
NO_SAMPLES = 10000

with open(data_folder + dataset+'/crf/'+str(NO_SAMPLES)+'_train_crf.txt') as f:
  data_json = json.load(f)
  train_feature = data_json["x"]
  y_train = data_json["y"]

In [83]:
print(train_feature[0])

[{'bias': 1.0, 'AbsolutePosition': 0, 'RelativePosition': 0.0, 'QuartilePosition': 0, '-1 BOS': True, '0:word.pos_': 'ADJ', '0:word.tag_': 'JJ', '0:word.dep_': 'amod', '0:word.ent_type': '', '0:word.ent_iob_': 'O', '0:word.lemma': 'presbyterian', '0:word.norm': 'presbyterian', '0:word.prefix1': 'P', '0:word.suffix1': 'n', '0:word.prefix2': 'Pr', '0:word.suffix2': 'an', '0:word.prefix3': 'Pre', '0:word.suffix3': 'ian', '0:srl_iob': 'O', '0:srl_type': '', '0:srl_verb': 'were', '0:morph_Degree': 'Pos', '0:word.is_alpha()': True, '0:word.like_num': False, '0:word.is_punct': False, '0:word.is_oov': True, '0:word.is_stop': False, '0:word.shape_reduced': 'Xx', '0:word.prob': -20.0, '1:word.pos_': 'NOUN', '1:word.tag_': 'NNS', '1:word.dep_': 'nsubjpass', '1:word.ent_type': '', '1:word.ent_iob_': 'O', '1:word.lemma': 'church', '1:word.norm': 'churches', '1:word.prefix1': 'c', '1:word.suffix1': 's', '1:word.prefix2': 'ch', '1:word.suffix2': 'es', '1:word.prefix3': 'chu', '1:word.suffix3': 'hes',

In [84]:
def flatten(list_of_lists):
    total_list = []
    for list_1 in list_of_lists:
        total_list += list_1
    return total_list

In [106]:
feature_name = '1:word.ent_iob_'
value_tuple = list(zip([x.get(feature_name, None) for x in flatten(train_feature)], flatten(y_train)))

In [107]:
from collections import Counter
value_counter = [element[0] for element in Counter([tup[0] for tup in value_tuple]).most_common()]

In [108]:
import plotly.express as px
px.histogram(x=[x[0] for x in value_tuple],
             facet_col=[x[1] for x in value_tuple],
             category_orders={"x": value_counter},
             title=feature_name,
             histnorm='percent')

In [110]:
value_counter

['O', 'B', 'I', None]

In [119]:
import spacy
nlp = spacy.load('en_core_web_trf')
sentence = nlp("This is a sentence with sadsad words")
print([token.is_oov for token in sentence])
print([token.prob for token in sentence])
print(nlp.Defaults.stop_words)

OSError: ignored

In [ ]:
#!python -m spacy download en_core_web_md